In [42]:
from mobile_coverage.data.load import get_data
import pandas as pd
from mobile_coverage import config
from keplergl import KeplerGl
import geopandas as gpd
import seaborn as sns
import matplotlib.pyplot as plt
from shapely import wkt
import plotly.express as px

In [2]:
np_df = get_data()

2026-01-19 19:31:29,349 [INFO] cell_coverage.build_data: len of full df: 25230402
2026-01-19 19:31:34,883 [INFO] cell_coverage.build_data: Number of cells with sufficient data: 4698
2026-01-19 19:31:36,163 [INFO] cell_coverage.build_data: Total number of cells: 4698
/home/tomf/Coding/MobileCoverage/crowd-source-mobile-coverage/mobile_coverage/data/load.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['month'] = pd.to_datetime(df['timestamp']).dt.to_period('M')


In [3]:
present_months = sorted(np_df['month'].unique())

train_months = present_months[:-1]
test_months = present_months[-1:]

print(f"Training months: {train_months}")
print(f"Testing month: {test_months}")

Training months: [Period('2025-03', 'M'), Period('2025-04', 'M'), Period('2025-05', 'M')]
Testing month: [Period('2025-06', 'M')]


In [4]:
hulls_df = pd.read_csv(config.HULLS_PATH)

In [17]:
results_df = pd.read_csv(config.MODEL_RESULTS_PATH)

results_df["geometry"] = results_df["geometry"].apply(wkt.loads)

results_df = gpd.GeoDataFrame(
    results_df,
    geometry="geometry",
    crs="EPSG:4326"  # change if your WKT is in a different CRS
)

In [25]:
results_df.head()

,cell_id,model_name,area_bin,n_points,area,geometry,point_precision,point_recall,point_f1,area_precision,area_recall,area_f1,hybrid_f1
0,fbde60e0d6def17690c655a27f33735b8fa006cd,convex_hull,1,1282,3.202272e-07,"POLYGON ((-1.84145 52.63412, -1.84149 52.63412...",1.0,0.955224,0.977099,0.920100,0.016713,0.032829,0.504964
1,fbde60e0d6def17690c655a27f33735b8fa006cd,svm,1,1282,3.202272e-07,"MULTIPOLYGON (((-1.84175 52.63409, -1.84176 52...",1.0,0.955224,0.977099,0.759523,0.019011,0.037093,0.507096
2,fbde60e0d6def17690c655a27f33735b8fa006cd,gmm,1,1282,3.202272e-07,"MULTIPOLYGON (((-1.84162 52.63398, -1.84169 52...",1.0,0.976119,0.987915,0.753627,0.016495,0.032283,0.510099
3,fbde60e0d6def17690c655a27f33735b8fa006cd,kde,1,1282,3.202272e-07,"MULTIPOLYGON (((-1.84162 52.63398, -1.8417 52....",1.0,0.973134,0.986384,0.751106,0.016534,0.032357,0.509370
4,fbde60e0d6def17690c655a27f33735b8fa006cd,cluster,1,1282,3.202272e-07,"MULTIPOLYGON (((-1.84231 52.634, -1.84217 52.6...",1.0,0.985075,0.992481,0.571098,0.017236,0.033461,0.512971


**Point-based precision** <br>
proportion of predicted-covered points that truly belong to the target cell (penalises spatial overreach).
<br> <br>
**Point-based recall** <br>
proportion of true cell points captured by the polygon (penalises under-coverage).
<br> <br>
**Point-based F1** <br>
harmonic mean of precision and recall, balancing omission and commission errors at the measurement level.
<br> <br>
**Area precision** <br>
fraction of the predicted polygon area that overlaps the ground-truth region (penalises overly large or diffuse boundaries).
<br> <br>
**Area recall** <br>
fraction of the ground-truth area recovered by the predicted polygon (penalises spatial under-coverage).
<br> <br>
**Area F1** <br>
harmonic mean of area precision and recall, measuring spatial agreement between predicted and true coverage regions.
<br> <br>

In [35]:
results_df

,cell_id,model_name,area_bin,n_points,area,geometry,point_precision,point_recall,point_f1,area_precision,area_recall,area_f1,hybrid_f1
0,fbde60e0d6def17690c655a27f33735b8fa006cd,convex_hull,1,1282,3.202272e-07,"POLYGON ((-1.84145 52.63412, -1.84149 52.63412...",1.0,0.955224,0.977099,0.920100,0.016713,0.032829,0.504964
1,fbde60e0d6def17690c655a27f33735b8fa006cd,svm,1,1282,3.202272e-07,"MULTIPOLYGON (((-1.84175 52.63409, -1.84176 52...",1.0,0.955224,0.977099,0.759523,0.019011,0.037093,0.507096
2,fbde60e0d6def17690c655a27f33735b8fa006cd,gmm,1,1282,3.202272e-07,"MULTIPOLYGON (((-1.84162 52.63398, -1.84169 52...",1.0,0.976119,0.987915,0.753627,0.016495,0.032283,0.510099
3,fbde60e0d6def17690c655a27f33735b8fa006cd,kde,1,1282,3.202272e-07,"MULTIPOLYGON (((-1.84162 52.63398, -1.8417 52....",1.0,0.973134,0.986384,0.751106,0.016534,0.032357,0.509370
4,fbde60e0d6def17690c655a27f33735b8fa006cd,cluster,1,1282,3.202272e-07,"MULTIPOLYGON (((-1.84231 52.634, -1.84217 52.6...",1.0,0.985075,0.992481,0.571098,0.017236,0.033461,0.512971
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2388,8c4ddc3a862eff3f0233c85135d94ee085083ce9,svm,9,4238,1.021200e-01,"MULTIPOLYGON (((-1.0689 52.73592, -1.06727 52....",1.0,0.748344,0.856061,0.998611,0.018597,0.036514,0.446287
2389,8c4ddc3a862eff3f0233c85135d94ee085083ce9,iso_forest,9,4238,1.021200e-01,"MULTIPOLYGON (((-1.02647 52.82636, -1.02634 52...",1.0,0.950331,0.974533,0.967187,0.373510,0.538905,0.756719
2390,8c4ddc3a862eff3f0233c85135d94ee085083ce9,gmm,9,4238,1.021200e-01,"MULTIPOLYGON (((-1.08732 52.78657, -1.08989 52...",1.0,0.970199,0.984874,0.771057,0.627040,0.691631,0.838253
2391,8c4ddc3a862eff3f0233c85135d94ee085083ce9,kde,9,4238,1.021200e-01,"MULTIPOLYGON (((-1.05212 52.7712, -1.05006 52....",1.0,0.788079,0.881481,0.992405,0.010107,0.020010,0.450746


In [44]:
fig = px.line(
    results_df,
    x="n_points",
    y="hybrid_f1",
    color="model_name",
    markers=True,
    line_group="model_name",
    hover_data=[
        "area_bin",
        "area",
        "point_f1",
        "area_f1"
    ],
)

fig.update_xaxes(type="log", title="Number of points (log scale)")
fig.update_yaxes(title="Hybrid F1")
fig.update_layout(
    title="Model performance vs data availability",
    legend_title="Model",
)

fig.show()


In [46]:
fig = px.box(
    results_df,
    x="area_bin",
    y="hybrid_f1",
    color="model_name",
    points="outliers",  # show individual cells
)

fig.update_layout(
    title="Model performance by coverage area",
    xaxis_title="Coverage area bin",
    yaxis_title="Hybrid F1",
)

fig.update_traces(jitter=0.3)

fig.show()

In [48]:
fig = px.scatter(
    results_df,
    x="area_precision",
    y="area_recall",
    color="model_name",
    symbol="area_bin",
    size="n_points",
    hover_data=[
        "cell_id",
        "hybrid_f1",
        "point_f1",
        "area"
    ],
)

fig.update_layout(
    title="Spatial precision–recall trade-off",
    xaxis_title="Area precision",
    yaxis_title="Area recall",
)

fig.show()

In [49]:
winners = (
    results_df.sort_values("hybrid_f1", ascending=False)
      .groupby(["area_bin", "n_points"], as_index=False)
      .first()
)

fig = px.scatter(
    winners,
    x="n_points",
    y="area_bin",
    color="model_name",
    size="hybrid_f1",
    hover_data=[
        "hybrid_f1",
        "point_f1",
        "area_f1",
        "area"
    ],
)

fig.update_xaxes(type="log")
fig.update_layout(
    title="Best-performing model by spatial regime",
    xaxis_title="Number of points (log scale)",
    yaxis_title="Area bin",
)

fig.show()

In [18]:
results_df['cell_id'].value_counts()

cell_id
8c4ddc3a862eff3f0233c85135d94ee085083ce9    6
a66317f94b47013ed2f7f8b16e08f34458d88ff1    6
80862a260b2df436c7d75375f8f79124c244048f    6
6cde5ecf2939c30c3cc025223b856af06fec059f    6
21a17eaff03ebc61c8e653def3c848dd89bf076f    6
                                           ..
78b8e8e2f28b2596924e2bf1af9339ab5091bcc1    5
87adfefd2275c325a69e2435e7dcafddd4beb8db    5
a2e08d0ed14f73559bc310f594f56d84ddff43f5    5
61e37ac4256c4d80c3ef757997f7ed86492228dc    5
fbde60e0d6def17690c655a27f33735b8fa006cd    5
Name: count, Length: 421, dtype: int64

In [31]:
cell_id = '61e37ac4256c4d80c3ef757997f7ed86492228dc'

np_fil = np_df[np_df['unique_cell'] == cell_id]
np_fil_train = np_fil[np_fil['month'].isin(train_months)]
np_fil_test = np_fil[np_fil['month'].isin(test_months)]

hull_fil = hulls_df[hulls_df['unique_cell'] == cell_id]
results_fil = results_df[results_df['cell_id'] == cell_id]

In [32]:
m = KeplerGl(height=600)
view_set = False

models = results_fil["model_name"].unique()

for model_name in models:
    row = results_fil.loc[results_fil["model_name"] == model_name]
    if row.empty:
        continue

    gdf = gpd.GeoDataFrame(
        row[["model_name", "point_f1"]].copy(),
        geometry=row.geometry.values,
        crs="EPSG:4326",
    )
    m.add_data(gdf, name=f"{model_name}_poly")

    if not view_set:
        centroid = gdf.geometry.iloc[0].centroid
        m.config = {
            "version": "v1",
            "config": {
                "mapState": {
                    "latitude": centroid.y,
                    "longitude": centroid.x,
                    "zoom": 12,
                    "pitch": 0,
                    "bearing": 0,
                }
            },
        }
        view_set = True
    
    m.add_data(data=np_fil_train, name="TrainPoints")
    m.add_data(data=np_fil_test, name="TestPoints")

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


/home/tomf/.cache/pypoetry/virtualenvs/mobile-coverage-WOMfuyQc-py3.12/lib/python3.12/site-packages/jupyter_client/session.py:727: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant: nan
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)
/home/tomf/.cache/pypoetry/virtualenvs/mobile-coverage-WOMfuyQc-py3.12/lib/python3.12/site-packages/jupyter_client/session.py:727: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant: nan
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)
/home/tomf/.cache/pypoetry/virtualenvs/mobile-coverage-WOMfuyQc-py3.12/lib/python3.12/site-packages/jupyter_client/session.py:727: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant: nan
Supporting this message i

In [33]:
m

KeplerGl(config={'version': 'v1', 'config': {'mapState': {'latitude': 52.76506170944345, 'longitude': -1.24199…